In [13]:
import numpy as np
import pandas as pd

In [14]:
ex = pd.read_excel('./実験用.xlsx', header=None)

display(ex.head(20))
print(len(ex))

,0,1
0,[表示値(1)],表示1
1,テーブル名,テーブル1
2,項目名,項目1
3,条件,条件1
4,NaN,NaN
5,[表示値(2)],表示2
6,テーブル名,テーブル2
7,項目名,項目2
8,条件,条件2
9,NaN,条件2


31


In [15]:
def parse_jmec(
    keys: list[str], values: list[str], primary_key: str, section_num: int
):

    # <-- INPUT VALIDATION -->
    if len(keys) != len(values):
        raise ValueError("Keys and values must have the same length")
    
    # <-- MAIN PROCESS -->
    # データのクリーニング
    clean_data: list[tuple[str, str]] = []
    str_list: list[str] = []
    for i in range(len(keys)):

        reversed_idx = len(keys) - i - 1

        # keysとvaluesの両方が空の場合はスキップ。
        if keys[reversed_idx] is np.nan and values[reversed_idx] is np.nan:
            continue
        # keysが空でvaluesが空でない場合は、valuesをスタックに積む。
        elif keys[reversed_idx] is np.nan and values[reversed_idx] is not np.nan:
            str_list.append(values[reversed_idx])
        # keysが空でなく、valuesが空の場合は、例外を投げる。
        elif keys[reversed_idx] is not np.nan and values[reversed_idx] is np.nan:
            raise Exception('index があるのに value がない')
        # keysとvaluesの両方がから出ない場合は、str_listを逆順に連結させる。
        elif keys[reversed_idx] is not np.nan and values[reversed_idx] is not np.nan:
            str_list.append(values[reversed_idx])
            str_list.reverse()
            string = ''.join(str_list)
            str_list = []
            clean_data.append([keys[reversed_idx], string])
        
    clean_data.reverse()
    print("DEBUG: clean_data")
    print(clean_data, end='\n\n')

    # primary_keyのidx取得
    pkey_indices = []
    for i in range(len(clean_data)):
        key = clean_data[i][0]
        if key.startswith(primary_key):
            pkey_indices.append(i)
    
    pkey_indices.append(len(clean_data))

    print("DEBUG: pkey_indices")
    print(pkey_indices, end='\n\n')

    # primary_keyのあとにいくつのセクションが続くのかカウント
    sectoin_cnts = []
    for i in range(1, len(pkey_indices)):
        prev = pkey_indices[i-1]
        curr = pkey_indices[i]

        cnt = (curr - prev - 1) // section_num
        sectoin_cnts.append(cnt)
    
    print("DEBUG: sectoin_cnts")
    print(sectoin_cnts, end='\n\n')

    # 新しい構造を作成
    ans = []
    for i in range(len(pkey_indices) - 1): # endを除くので-1
        for j in range(sectoin_cnts[i]):

            # 1行目をpkeyで始める
            ans_row = [clean_data[pkey_indices[i]][1]]

            for k in range(section_num):

                # セクションの要素を追加していく
                idx = pkey_indices[i] + section_num * j + k + 1
                ans_row.append(clean_data[idx][1])
            
            ans.append(ans_row)
    
    print("DEBUG: ans")
    print(ans, end='\n\n')

    return ans


ans = parse_jmec(ex[0].tolist(), ex[1].tolist(), '[表示値', 3)

DEBUG: clean_data
[['[表示値(1)]', '表示1'], ['テーブル名', 'テーブル1'], ['項目名', '項目1'], ['条件', '条件1'], ['[表示値(2)]', '表示2'], ['テーブル名', 'テーブル2'], ['項目名', '項目2'], ['条件', '条件2条件2条件2'], ['[表示値(3)]', '表示3'], ['テーブル名', 'テーブル3.1'], ['項目名', '項目3.1'], ['条件', '条件3.1'], ['テーブル名', 'テーブル3.2'], ['項目名', '項目3.2'], ['条件', '条件3.2'], ['[表示値(4)]', '表示4'], ['テーブル名', 'テーブル4.1'], ['項目名', '項目4.1'], ['条件', '条件4.1条件4.1'], ['テーブル名', 'テーブル4.2'], ['項目名', '項目4.2'], ['条件', '条件4.2条件4.2']]

DEBUG: pkey_indices
[0, 4, 8, 15, 22]

DEBUG: sectoin_cnts
[1, 1, 2, 2]

DEBUG: ans
[['表示1', 'テーブル1', '項目1', '条件1'], ['表示2', 'テーブル2', '項目2', '条件2条件2条件2'], ['表示3', 'テーブル3.1', '項目3.1', '条件3.1'], ['表示3', 'テーブル3.2', '項目3.2', '条件3.2'], ['表示4', 'テーブル4.1', '項目4.1', '条件4.1条件4.1'], ['表示4', 'テーブル4.2', '項目4.2', '条件4.2条件4.2']]



In [16]:
df = pd.DataFrame(ans)
display(df)

,0,1,2,3
0,表示1,テーブル1,項目1,条件1
1,表示2,テーブル2,項目2,条件2条件2条件2
2,表示3,テーブル3.1,項目3.1,条件3.1
3,表示3,テーブル3.2,項目3.2,条件3.2
4,表示4,テーブル4.1,項目4.1,条件4.1条件4.1
5,表示4,テーブル4.2,項目4.2,条件4.2条件4.2
